In [100]:
import pandas as pd
from datetime import datetime

import yfinance as yf

import plotly.graph_objects as go

In [101]:
# df = pd.read_csv("./data/EURUSD=X.csv", parse_dates=True)

df = yf.download("EURUSD=X", period="60d", interval="15m")

date = df.index

df['Date'] = date

print(df.head())

df['ema55'] = df['Close'].ewm(span=55).mean()
df['ema200'] = df['Close'].ewm(span=200).mean()

df['ema_diff'] = df['ema55'] - df['ema200']
df['crossover_type'] = df['ema_diff'].apply(
    lambda x: 'bullish' if x > 0 else 'bearish')


df = df.sort_values(by='Date')
df = df.dropna()
df = df.reset_index(drop=True)


[*********************100%***********************]  1 of 1 completed
                         Open      High       Low     Close  Adj Close  \
Datetime                                                                 
2023-01-05 00:00:00  1.060670  1.061684  1.060670  1.061346   1.061346   
2023-01-05 00:15:00  1.061458  1.062812  1.061458  1.062812   1.062812   
2023-01-05 00:30:00  1.062699  1.063038  1.062248  1.062699   1.062699   
2023-01-05 00:45:00  1.062699  1.063490  1.062699  1.063264   1.063264   
2023-01-05 01:00:00  1.063151  1.063490  1.062925  1.063038   1.063038   

                     Volume                Date  
Datetime                                         
2023-01-05 00:00:00       0 2023-01-05 00:00:00  
2023-01-05 00:15:00       0 2023-01-05 00:15:00  
2023-01-05 00:30:00       0 2023-01-05 00:30:00  
2023-01-05 00:45:00       0 2023-01-05 00:45:00  
2023-01-05 01:00:00       0 2023-01-05 01:00:00  


In [102]:
fig = go.Figure(
    data=[go.Candlestick(x=df['Date'],  open=df['Open'], high=df['High'], low=df['Low'], close=df['Close']),
          go.Scatter(x=df['Date'], y=df['ema55'], name='ema55'),
          go.Scatter(x=df['Date'], y=df['ema200'], name='ema200')
          ])


fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()


In [103]:
start_cash = 1000
trades = pd.DataFrame(columns=[
    'entry_date', 'exit_date', 'entry_price', 'exit_price', 'crossover_type', 'pnL', 'target', 'sl', 'balance'
])

position = {
    'entry_date': None,
    'exit_date': None,
    'entry_price': None,
    'exit_price': None,
    'is_long': False,
    'is_short': False,
    'active': False,
    'crossover_type': None,
    'target': None,
    'sl': None,
    'size': None,
    'balance': None,
    'pnL': None
}


def clear_position():
    position['entry_date'] = None
    position['exit_date'] = None
    position['entry_price'] = None
    position['exit_price'] = None
    position['is_long'] = False
    position['is_short'] = False
    position['active'] = False
    position['start_crossover_type'] = None
    position['target'] = None
    position['sl'] = None


def open(row, size=1, sl=0, target=0):
    if row is None:
        raise Exception("Provide opening trade data")

    position['entry_date'] = row['Date']
    position['entry_price'] = row['Close']
    position['is_long'] = row['crossover_type'] == 'bullish'
    position['is_short'] = row['crossover_type'] == 'bearish'
    position['active'] = True
    position['crossover_type'] = row['crossover_type']
    position['target'] = target
    position['sl'] = sl
    position['size'] = size
    position['balance'] = start_cash

    cost = position['entry_price'] * position['size']
    balance_amount = start_cash - cost

    if balance_amount < 0:
        raise Exception("Not enough balance to open position")

    position['cost'] = cost
    position['balance'] = balance_amount


def close(row):
    if row is None:
        raise Exception("Provide closing trade data")

    if not position['active']:
        raise Exception("Cannot close an inactive position")

    position['exit_date'] = row['Date']
    position['exit_price'] = row['Close']
    position['pnL'] = (position['exit_price'] * position['size'] -
                       position['entry_price']) * position['size']

    if (position['is_short']):
        position['pnL'] = -position['pnL']

    position['balance'] = position['balance'] + position['pnL']

    global trades
    t_df = pd.DataFrame([position])
    trades = pd.concat([trades, t_df], ignore_index=True)
    clear_position()


df['position'] = None


In [104]:
start = 200
trades = pd.DataFrame(None)

while start < len(df):

    if position['active']:

        candle = df.iloc[start]

        # diff = candle['Close'] - position['entry_price']
        # print(pd.Series(position))

        candle_close = candle['Close']
        entry_price = position['entry_price']

        if position['is_long']:
            target_price = entry_price + position['target']
            stoploss = round(entry_price - position['sl'], 5)

            print("[Long]", entry_price, target_price, stoploss,
                  candle_close)

            if candle_close >= target_price or candle_close <= stoploss:
                close(candle)

        if position['is_short']:
            target_price = entry_price - position['target']
            stoploss = round(entry_price + position['sl'], 5)

            print("[Short]", entry_price, target_price, stoploss,
                  candle_close)

            if (candle_close <= target_price or candle_close >= entry_price):
                close(candle)

           # Skip to next row if position is active
        start = start + 1
        continue

    if df['crossover_type'][start] != df['crossover_type'][start-1]:
        df['position'][start] = 'crossover'

        # print(df.iloc[start])

        open(df.iloc[start], target=0.004, sl=0.001, size=1)

    start = start + 1

print(start)


[Short] 1.078399658203125 1.074399658203125 1.0794 1.078515887260437
[Short] 1.078399658203125 1.074399658203125 1.0794 1.079214334487915
[Short] 1.078399658203125 1.074399658203125 1.0794 1.079097867012024
[Short] 1.078399658203125 1.074399658203125 1.0794 1.0794472694396973
[Short] 1.078399658203125 1.074399658203125 1.0794 1.0797970294952393
[Short] 1.078399658203125 1.074399658203125 1.0794 1.0797970294952393
[Short] 1.078399658203125 1.074399658203125 1.0794 1.0803803205490112
[Short] 1.078399658203125 1.074399658203125 1.0794 1.0806137323379517
[Short] 1.078399658203125 1.074399658203125 1.0794 1.079563856124878
[Short] 1.078399658203125 1.074399658203125 1.0794 1.079563856124878
[Short] 1.078399658203125 1.074399658203125 1.0794 1.0793308019638062
[Short] 1.078399658203125 1.074399658203125 1.0794 1.079097867012024
[Short] 1.078399658203125 1.074399658203125 1.0794 1.0796804428100586
[Short] 1.078399658203125 1.074399658203125 1.0794 1.079563856124878
[Short] 1.078399658203125 1

/tmp/ipykernel_26053/4268464301.py:45: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_26053/4268464301.py:45: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



TypeError: close() missing 1 required positional argument: 'exit_price'

In [ ]:
print("Total PnL: ", trades['pnL'].sum())

Total PnL:  0.020125985145568848
